In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('dataset1.csv', index_col='invoice_id')
df2 = pd.read_csv('dataset2.csv')

In [3]:
#join two datasets and get rate
df_combined = pd.merge(df1, df2, how='inner', on='counter_party')

In [4]:
#Get max rating by counter_party
grouped_counterparty = df_combined.groupby('counter_party')
counterparty_max_rating = pd.DataFrame(grouped_counterparty['rating'].agg(np.max))
counterparty_max_rating = pd.concat([counterparty_max_rating, pd.DataFrame({'rating': 'N/A'}, index=['Total'])])
counterparty_max_rating.reset_index(inplace=True)
counterparty_max_rating = counterparty_max_rating.rename(columns= {'index':'counter_party', 'rating': 'max(rating by counterparty)'})
counterparty_max_rating

,counter_party,max(rating by counterparty)
0,C1,3
1,C2,3
2,C3,6
3,C4,6
4,C5,6
5,C6,6
6,Total,N/A


In [5]:
#split into ARAP and ACCR
arap_dataset = df_combined[df_combined['status'] == 'ARAP']
accr_dataset = df_combined[df_combined['status'] == 'ACCR']

In [6]:
#get group by le records
le_group_by_arap = arap_dataset.groupby('legal_entity')['value'].sum().reset_index().rename(columns={'index': 'legal_entity', 'value': 'sum(value where status = ARAP)'})
le_group_by_accr = accr_dataset.groupby('legal_entity')['value'].sum().reset_index().rename(columns={'index': 'legal_entity', 'value': 'sum(value where status = ACCR)'})
le_group_by = pd.merge(le_group_by_arap, le_group_by_accr, how='inner', on='legal_entity')
le_group_by["counter_party"] = 'Total'
le_group_by["tier"] = 'Total'
le_group_by["max(rating by counterparty)"] = 'N/A'
le_group_by

,legal_entity,sum(value where status = ARAP),sum(value where status = ACCR),counter_party,tier,max(rating by counterparty)
0,L1,85,100,Total,Total,N/A
1,L2,1020,207,Total,Total,N/A
2,L3,145,205,Total,Total,N/A


In [7]:
#get group by le, cp records
le_cp_group_by_arap = arap_dataset.groupby(['legal_entity', 'counter_party'])['value'].sum().reset_index().rename(columns={'index': 'legal_entity', 'value': 'sum(value where status = ARAP)'})
le_cp_group_by_accr = accr_dataset.groupby(['legal_entity', 'counter_party'])['value'].sum().reset_index().rename(columns={'index': 'legal_entity', 'value': 'sum(value where status = ACCR)'})
le_cp_group_by = pd.merge(le_cp_group_by_arap, le_cp_group_by_accr, how='outer', on=['legal_entity', 'counter_party'])
le_cp_group_by["tier"] = 'Total'
le_cp_group_by = pd.merge(le_cp_group_by, counterparty_max_rating, how='inner', on='counter_party')
le_cp_group_by

,legal_entity,counter_party,sum(value where status = ARAP),sum(value where status = ACCR),tier,max(rating by counterparty)
0,L1,C1,40.0,NaN,Total,3
1,L1,C3,5.0,NaN,Total,6
2,L2,C3,NaN,52.0,Total,6
3,L3,C3,NaN,145.0,Total,6
4,L1,C4,40.0,100.0,Total,6
5,L2,C2,20.0,40.0,Total,3
6,L2,C5,1000.0,115.0,Total,6
7,L3,C6,145.0,60.0,Total,6


In [8]:
#get group by cp records
cp_group_by_arap = arap_dataset.groupby('counter_party')['value'].sum().reset_index().rename(columns={'index': 'counter_party', 'value': 'sum(value where status = ARAP)'})
cp_group_by_accr = accr_dataset.groupby('counter_party')['value'].sum().reset_index().rename(columns={'index': 'counter_party', 'value': 'sum(value where status = ACCR)'})
cp_group_by = pd.merge(cp_group_by_arap, cp_group_by_accr, how='outer', on='counter_party')
cp_group_by['legal_entity'] = 'Total'
cp_group_by['tier'] = 'Total'
cp_group_by = pd.merge(cp_group_by, counterparty_max_rating, how='inner', on='counter_party')
cp_group_by

,counter_party,sum(value where status = ARAP),sum(value where status = ACCR),legal_entity,tier,max(rating by counterparty)
0,C1,40,NaN,Total,Total,3
1,C2,20,40.0,Total,Total,3
2,C3,5,197.0,Total,Total,6
3,C4,40,100.0,Total,Total,6
4,C5,1000,115.0,Total,Total,6
5,C6,145,60.0,Total,Total,6


In [9]:
#get group by tier records
ti_group_by_arap = arap_dataset.groupby('tier')['value'].sum().reset_index().rename(columns={'index': 'tier', 'value': 'sum(value where status = ARAP)'})
ti_group_by_accr = accr_dataset.groupby('tier')['value'].sum().reset_index().rename(columns={'index': 'tier', 'value': 'sum(value where status = ACCR)'})
ti_group_by = pd.merge(ti_group_by_arap, ti_group_by_accr, how='outer', on='tier')
ti_group_by['legal_entity'] = 'Total'
ti_group_by['counter_party'] = 'Total'
ti_group_by["max(rating by counterparty)"] = 'N/A'
ti_group_by

,tier,sum(value where status = ARAP),sum(value where status = ACCR),legal_entity,counter_party,max(rating by counterparty)
0,1,40,NaN,Total,Total,N/A
1,2,20,40.0,Total,Total,N/A
2,3,5,197.0,Total,Total,N/A
3,4,40,100.0,Total,Total,N/A
4,5,1000,115.0,Total,Total,N/A
5,6,145,60.0,Total,Total,N/A


In [10]:
#combine totals
combined_group_by_totals = pd.concat([le_group_by, le_cp_group_by, cp_group_by, ti_group_by], ignore_index=True)
combined_group_by_totals

,legal_entity,sum(value where status = ARAP),sum(value where status = ACCR),counter_party,tier,max(rating by counterparty)
0,L1,85.0,100.0,Total,Total,N/A
1,L2,1020.0,207.0,Total,Total,N/A
2,L3,145.0,205.0,Total,Total,N/A
3,L1,40.0,NaN,C1,Total,3
4,L1,5.0,NaN,C3,Total,6
5,L2,NaN,52.0,C3,Total,6
6,L3,NaN,145.0,C3,Total,6
7,L1,40.0,100.0,C4,Total,6
8,L2,20.0,40.0,C2,Total,3
9,L2,1000.0,115.0,C5,Total,6


In [11]:
#output to file
combined_group_by_totals.to_csv('out_pandas.csv', index=False, columns=['legal_entity', 'counter_party', 'tier', 'max(rating by counterparty)', 'sum(value where status = ARAP)', 'sum(value where status = ACCR)'], na_rep=0.0)